## TF Files

In [ ]:
from random import shuffle
import glob
import sys
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,BatchNormalization,Activation,Dropout,AveragePooling2D,MaxPooling2D,Flatten,Dense,Input,Concatenate
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model

In [3]:
tf.__version__

'1.14.0'

In [ ]:
tf.enable_eager_execution()

In [ ]:
#write a function to download the dataset from tenserflow datasets
from tensorflow.keras.datasets import cifar10
from sklearn.utils import shuffle

def get_data(x,preprocess=False):
  (X_train, y_train), (X_test, y_test) = x.load_data()
  num_classes = len(np.unique(y_train))
  X_train, Y_train = shuffle(X_train, y_train)
  X_test, Y_test = shuffle(X_test, y_test)

  if preprocess:
        X_train = X_train.astype('float32')
        X_test = X_test.astype('float32')
        X_train /= 255
        X_test /= 255
  
  #Y_train = to_categorical(y_train, num_classes)
  #Y_test = to_categorical(y_test, num_classes)
  return X_train, Y_train, X_test, Y_test

In [6]:

X_train, Y_train, X_test, Y_test=get_data(cifar10)
print(Y_train.shape)
print(type(Y_train))
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]
len_train, len_test = len(X_train), len(X_test)
Y_train = Y_train.astype('int64').reshape(len_train)
print(Y_train.shape)
print(type(Y_train))
Y_test = Y_test.astype('int64').reshape(len_test)

170500096/170498071 [==============================] - 9s 0us/step
(50000, 1)
<class 'numpy.ndarray'>
(50000,)
<class 'numpy.ndarray'>


In [ ]:
import tensorflow as tf
import numpy as np

def _bytes_feature(value):
	"""Returns a bytes_list from a string / byte."""
	return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _float_feature(value):
	"""Returns a float_list from a float / double."""
	return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def _int64_feature(value):
	"""Returns an int64_list from a bool / enum / int / uint."""
	return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def serialize_function(x, y):
	example = tf.train.Example(features=tf.train.Features(
					feature={
							'image': _bytes_feature(tf.compat.as_bytes(x.tostring())),
							'label': _int64_feature(int(y))
					}))
	return example.SerializeToString()


def convert_to_tfrecord(data_set, file_name):
	with tf.python_io.TFRecordWriter(file_name) as record_writer:
		for x, y in data_set:
			if isinstance(x, (np.ndarray, np.generic)) and isinstance(y, (np.ndarray, np.generic)):
				record = serialize_function(x, y)
			else:
				record = serialize_function(x.numpy(), y.numpy())
			record_writer.write(record)
		print(f"TFRecord id created at path : '{file_name}', Done")


def parser(record, shape=[32, 32, 3], num_of_class=10):
	feature_description = {
	'image': tf.FixedLenFeature([], tf.string, default_value=''),
	'label': tf.FixedLenFeature([], tf.int64, default_value=0)
	# 'feature2': tf.FixedLenFeature([], tf.string, default_value=''),
	# 'feature3': tf.FixedLenFeature([], tf.float32, default_value=0.0),
		}
	parsed = tf.parse_single_example(record, feature_description)
	image = tf.decode_raw(parsed["image"], tf.uint8)
	image = tf.cast(image, tf.float32)
	image = tf.reshape(image, shape=shape)
	label = tf.cast(parsed["label"], tf.int32)
	label = tf.one_hot(label, num_of_class, dtype=tf.int32)
	return image, label


class CreateTFRecord(object):
    """docstring for CreateTFRecord"""
    '''
    EXP.
    data_to_write = zip(x_train, y_train)
    file name = 'train.tfrecord'
    '''
    def __init__(self, data_to_write, file_name):
        super().__init__()
        self.data_to_write = data_to_write
        self.file_name = file_name
        convert_to_tfrecord(self.data_to_write, self.file_name)

class Get_dataset(object):
	def __init__(self, file_name , buffer_size=1024, seed=1, batch_size=32, GPU_buffer_size = None):
		self.file_name = file_name
		self.buffer_size=buffer_size
		self.seed=seed
		self.batch_size=batch_size
		self.GPU_buffer_size = GPU_buffer_size
		input_foo(self.file_name, self.buffer_size, self.seed, self.batch_size, self.GPU_buffer_size)
# CreateTFRecord("kk.tfrecord", "kk")

## TFRecords

In [8]:
# installing data_science_utils repo.
pip install --upgrade --upgrade-strategy only-if-needed https://github.com/kkrunal77/data_science_utils/tarball/master


     - 10kB 11.1MB/s
     |████████████████████████████████| 2.3MB 41.6MB/s 
  Created wheel for data-science-utils: filename=data_science_utils-0.0.0-cp36-none-any.whl size=3777 sha256=de2c99c15ee573d8bc716357ca576359ddf7c90376e132bfb7b987331e856cff
  Stored in directory: /tmp/pip-ephem-wheel-cache-x41brhk6/wheels/72/1a/9f/4d1aadec4afb9693f42c521c5be90d606f56f435ba21e38756
Successfully built data-science-utils


In [9]:
from data.imports import *

TFrecord.CreateTFRecord(zip(X_train, Y_train), 'train.tfrecord')
TFrecord.CreateTFRecord(zip(X_test, Y_test), 'test.tfrecord')

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.


TFRecord id created at path : 'train.tfrecord', Done
TFRecord id created at path : 'test.tfrecord', Done


In [10]:
8*64*64

32768

In [ ]:
#tuning pipline

def input_foo(filenames, buffer_size, seed, batch_size, GPU_buffer_size):
    
	dataset = tf.data.TFRecordDataset(filenames=filenames, buffer_size=buffer_size)

	dataset = dataset.apply(
		tf.data.experimental.shuffle_and_repeat(buffer_size=buffer_size,
										   seed=seed)
		)
	dataset = dataset.apply(
		tf.data.experimental.map_and_batch(
								  map_func=parser,
								  batch_size=batch_size,
								  num_parallel_calls=tf.data.experimental.AUTOTUNE)
		)
	dataset = dataset.apply(tf.data.experimental.prefetch_to_device('/GPU:0', buffer_size=GPU_buffer_size))
	dataset = dataset.prefetch(8)
	return dataset

In [ ]:
# test_dataset = TFrecord.input_foo(filenames='test.tfrecord', buffer_size=1024, seed=2, batch_size=64, GPU_buffer_size=None)
# train_dataset = TFrecord.input_foo(filenames='train.tfrecord', buffer_size=1024, seed=2, batch_size=64, GPU_buffer_size=None)

test_dataset = input_foo(filenames='test.tfrecord', buffer_size=8*128*128, seed=50, batch_size=64, GPU_buffer_size=None)
train_dataset = input_foo(filenames='train.tfrecord', buffer_size=8*128*128, seed=50, batch_size=64, GPU_buffer_size=None)

##Training

In [15]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

Using TensorFlow backend.


In [ ]:
# Hyperparameters
batch_size = 64 
num_classes = 10
epochs = 50
l = 6 
num_filter = 32
compression = 0.5
dropout_rate = 0.2

In [ ]:
# Dense Block
def add_denseblock(input, num_filter = 64, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
            Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [ ]:
def add_transition(input, num_filter = 32, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
        Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [ ]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [ ]:
num_filter = 32
dropout_rate = 0.2
l = 6
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

#Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
#Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Second_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [21]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 32)   128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
class stop_at_Acc(tf.keras.callbacks.Callback):
    def __init__(self,validation_iterator, validation_steps, threshold = 0.85, ):
        super(stop_at_Acc, self).__init__()
        self.threshold = threshold
        self.validation_iterator = validation_iterator
        self.validation_steps = validation_steps

    def on_epoch_end(self, epoch, logs={}):
        # score = self.model.evaluate_generator(self.validation_iterator,steps=len(self.validation_iterator)) #use for genrator
        score = model.evaluate(self.validation_iterator,steps=self.validation_steps, verbose=0)
        acc = score[1]
        if acc >= self.threshold:
            self.model.stop_training = True
            print("Stopping Training:: Val Acc : %.3f Achieved"%(acc))
            
stopper = stop_at_Acc(test_dataset, validation_steps=len_test//batch_size, threshold = 0.85)

In [ ]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.01,momentum=0.9),
              metrics=['accuracy'])

In [25]:
# print(train_input_fn())
print(train_dataset)
print(test_dataset)

<DatasetV1Adapter shapes: ((?, 32, 32, 3), (?, 10)), types: (tf.float32, tf.int32)>
<DatasetV1Adapter shapes: ((?, 32, 32, 3), (?, 10)), types: (tf.float32, tf.int32)>


In [26]:
model.fit(
            train_dataset,
            epochs=75,
            steps_per_epoch = len_train//batch_size,
            # validation_data=test_dataset,
            # validation_steps=len_test//batch_size,
            verbose = 1,
            callbacks = [stopper]
                    )
#log seed = 64, prefetch=10, buffer_size=8*1024*1024, no descrease (39 sec)

Epoch 1/75
781/781 [==============================] - 69s 89ms/step - loss: 1.4173 - acc: 0.4907
Epoch 2/75
781/781 [==============================] - 53s 67ms/step - loss: 1.0321 - acc: 0.6353
Epoch 3/75
781/781 [==============================] - 53s 67ms/step - loss: 0.8628 - acc: 0.6953
Epoch 4/75
781/781 [==============================] - 53s 68ms/step - loss: 0.7665 - acc: 0.7314
Epoch 5/75
781/781 [==============================] - 53s 68ms/step - loss: 0.6976 - acc: 0.7545
Epoch 6/75
781/781 [==============================] - 53s 68ms/step - loss: 0.6464 - acc: 0.7728
Epoch 7/75
781/781 [==============================] - 53s 67ms/step - loss: 0.6050 - acc: 0.7876
Epoch 8/75
781/781 [==============================] - 53s 68ms/step - loss: 0.5722 - acc: 0.7991
Epoch 9/75
781/781 [==============================] - 53s 68ms/step - loss: 0.5475 - acc: 0.8098
Epoch 10/75
781/781 [==============================] - 53s 68ms/step - loss: 0.5206 - acc: 0.8178
Epoch 11/75
781/781 [========

In [ ]:
#log seed = 50, prefetch=4, reduce by 1/2

Epoch 1/75
781/781 [==============================] - 43s 55ms/step - loss: 1.4720 - acc: 0.4687
Epoch 2/75
781/781 [==============================] - 38s 49ms/step - loss: 1.0808 - acc: 0.6158
Epoch 3/75
781/781 [==============================] - 38s 48ms/step - loss: 0.8911 - acc: 0.6861
Epoch 4/75
781/781 [==============================] - 38s 49ms/step - loss: 0.7853 - acc: 0.7259
Epoch 5/75
781/781 [==============================] - 38s 49ms/step - loss: 0.7189 - acc: 0.7479
Epoch 6/75
781/781 [==============================] - 38s 49ms/step - loss: 0.6701 - acc: 0.7651
Epoch 7/75
781/781 [==============================] - 38s 49ms/step - loss: 0.6266 - acc: 0.7799
Epoch 8/75
781/781 [==============================] - 38s 49ms/step - loss: 0.5945 - acc: 0.7911
Epoch 9/75
781/781 [==============================] - 38s 49ms/step - loss: 0.5664 - acc: 0.8016
Epoch 10/75
781/781 [==============================] - 38s 49ms/step - loss: 0.5395 - acc: 0.8132
Epoch 11/75
781/781 [========

In [ ]:
#log for seed=2, prefetch=2

Epoch 1/75
781/781 [==============================] - 88s 112ms/step - loss: 0.5232 - acc: 0.8164
Epoch 2/75
781/781 [==============================] - 84s 108ms/step - loss: 0.5047 - acc: 0.8226
Epoch 3/75
781/781 [==============================] - 85s 109ms/step - loss: 0.4876 - acc: 0.8278
Epoch 4/75
156/156 [==============================] - 4s 29ms/step - loss: 0.4296 - acc: 0.8512
Stopping Training:: Val Acc : 0.851 Achieved
781/781 [==============================] - 84s 108ms/step - loss: 0.4733 - acc: 0.8356


In [ ]:
# Test the model
score = model.evaluate(test_dataset, steps=len_test//batch_size)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

 19/156 [==>...........................] - ETA: 5:50 - loss: 0.6259 - acc: 0.8150

In [ ]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

In [ ]:
from google.colab import files

files.download('DNST_model.h5')

# for time comparision(without TFRecord)

In [ ]:
class stop_at_Acc(tf.keras.callbacks.Callback):
    def __init__(self,validation_iterator, threshold = 0.85, ):
        super(stop_at_Acc, self).__init__()
        self.threshold = threshold
        self.validation_iterator = validation_iterator
        
    def on_epoch_end(self, epoch, logs={}):
        score = self.model.evaluate_generator(self.validation_iterator, steps=len(self.validation_iterator))
        acc = score[1]
        if acc >= self.threshold:
            self.model.stop_training = True
            print("Stopping Training:: Val Acc : %.3f Achieved"%(acc))
            
stopper = stop_at_Acc(validation_iterator, threshold = 0.80)

In [ ]:
X_train.shape

(50000, 32, 32, 3)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

datagen = ImageDataGenerator()
datagen.fit(X_train)

y_train = to_categorical(Y_train, num_classes=10, dtype='float32')
y_test = to_categorical(Y_test, num_classes=10, dtype='float32')
datagen_validation = ImageDataGenerator(featurewise_center=False,
                                        featurewise_std_normalization=False,
                                        )


validation_iterator = datagen_validation.flow(X_test, y_test, batch_size=batch_size,shuffle=True)

train_iterator = datagen.flow(X_train, y_train, batch_size=batch_size, shuffle=True)


In [ ]:
len_train

50000

In [ ]:
model.fit_generator(train_iterator,
                    steps_per_epoch=len_train//batch_size, 
                    validation_data = validation_iterator, 
                    validation_steps = len_test//batch_size,
                    epochs=2,
                    verbose=1,
                    # callbacks=[stopper]
                    )

Epoch 1/2
781/781 [==============================] - 97s 124ms/step - loss: 0.7160 - acc: 0.7494 - val_loss: 1.1623 - val_acc: 0.6544
Epoch 2/2
781/781 [==============================] - 97s 124ms/step - loss: 0.6649 - acc: 0.7663 - val_loss: 0.8300 - val_acc: 0.7323


In [ ]:
score = model.evaluate_generator(validation_iterator, steps=len(validation_iterator))

In [ ]:
model.fit(X_train,y_train,batch_size=64, epochs=2)

Epoch 1/2
50000/50000 [==============================] - 98s 2ms/sample - loss: 1.4700 - acc: 0.4740
Epoch 2/2
50000/50000 [==============================] - 90s 2ms/sample - loss: 1.0425 - acc: 0.6300
